# BUSINESS UNDERSTANDING

Benefit of being able to help people sift through myriad availibilities. 

# DATA UNDERSTANDING

What does the data look like?

What information can we glean?

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

In [9]:
df = spark.read.csv('data/ratings.csv', header=True)

In [13]:
df.count()
df.

26024289

# DATA PREPARATION

Train Test Split

# MODELING

PySpark Alternating Least Squares

# EVALUATION

Would evaluate in ... manner.

# DEPLOYMENT

Would deploy in ... manner.